In [43]:
from math import isclose
from time import sleep

import numpy as np
import plotly.graph_objects as go
from numpy._typing import NDArray
from numpy.testing import assert_equal
from tqdm import trange
from zema_emc_annotated.dataset import ZeMASamples  # type: ignore[import]

from lp_nn_robustness_verification.data_acquisition.activation_functions import (
    Sigmoid,
)
from lp_nn_robustness_verification.data_acquisition.generate_nn_params import (
    construct_out_features_counts,
    generate_weights_and_biases,
)
from lp_nn_robustness_verification.data_acquisition.uncertain_inputs import (
    UncertainInputs,
)
from lp_nn_robustness_verification.data_types import (
    UncertainArray,
    ValidCombinationForZeMA,
)
from lp_nn_robustness_verification.linear_program import RobustnessVerification
from lp_nn_robustness_verification.pre_processing import LinearInclusion

## Preparations

Set up shared Plotly layout parameters.

In [2]:
bg_color = "#1E1F22"
shared_layout = dict(
    paper_bgcolor=bg_color,
    font=dict(
        size=20,
    ),
    xaxis_title="1st component",
    yaxis_title="2nd component",
    autosize=False,
    width=1300,
    height=1300,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=1.05,
        xanchor="left",
        x=0.01,
        font=dict(
            size=30,
        ),
    ),
    plot_bgcolor=bg_color,
)
shared_ratio = dict(
    scaleanchor="x",
    scaleratio=1,
)
shared_title_params = dict(
    y=0.9,
    x=0.5,
    xanchor="center",
    yanchor="top",
)

## Run batch of examples with varying random seed

valid seeds are:

- ValidCombinationForZeMA(size_scaler=1, depth=1): 2
- ValidCombinationForZeMA(size_scaler=1, depth=3): 2123
- ValidCombinationForZeMA(size_scaler=1, depth=5): 80986
- ValidCombinationForZeMA(size_scaler=1, depth=8): 80986
- ValidCombinationForZeMA(size_scaler=10, depth=1): 0

In [ ]:
valid_seeds: dict[ValidCombinationForZeMA, int] = {}
size_scalers = [2000]
depths = [8]
for size_scaler in size_scalers:
    zema_data = ZeMASamples(1, size_scaler=size_scaler, normalize=True)
    uncertain_inputs = UncertainInputs(
        UncertainArray(zema_data.values[0], zema_data.uncertainties[0])
    )
    for depth in depths:
        print(
            f"Trying to find valid seed for {ValidCombinationForZeMA(size_scaler, depth)}"
        )
        for seed in trange(100000):
            linear_inclusion = LinearInclusion(
                uncertain_inputs,
                Sigmoid,
                generate_weights_and_biases(
                    len(uncertain_inputs.values),
                    construct_out_features_counts(
                        len(uncertain_inputs.values), depth=depth
                    ),
                    seed,
                ),
            )
            optimization = RobustnessVerification(linear_inclusion)
            optimization.model.hideOutput()
            optimization.solve()
            if optimization.model.getSols():
                valid_seeds[ValidCombinationForZeMA(size_scaler, depth)] = seed
                break
sleep(0.5)
print(f"valid seeds: {valid_seeds}")

    Extract values from /ZeMA_DAQ/Acceleration/qudt:value
    Values extracted
    Extract uncertainties from /ZeMA_DAQ/Acceleration/qudt:standardUncertainty
    Uncertainties extracted
    Extract values from /ZeMA_DAQ/Active_Current/qudt:value
    Values extracted
    Extract uncertainties from /ZeMA_DAQ/Active_Current/qudt:standardUncertainty
    Uncertainties extracted
    Extract values from /ZeMA_DAQ/Force/qudt:value
    Values extracted
    Extract uncertainties from /ZeMA_DAQ/Force/qudt:standardUncertainty
    Uncertainties extracted
    Extract values from /ZeMA_DAQ/Motor_Current/qudt:value
    Values extracted
    Extract uncertainties from /ZeMA_DAQ/Motor_Current/qudt:standardUncertainty
    Uncertainties extracted
    Extract values from /ZeMA_DAQ/Pressure/qudt:value
    Values extracted
    Extract uncertainties from /ZeMA_DAQ/Pressure/qudt:standardUncertainty
    Uncertainties extracted
    Extract values from /ZeMA_DAQ/Sound_Pressure/qudt:value
    Values extracted
    E


  0%|          | 0/100000 [00:00<?, ?it/s]

## Solution to the linear optimization problem

In [23]:
zema_data = ZeMASamples(1, size_scaler=1, normalize=True, idx_start=0)
uncertain_inputs = UncertainInputs(
    UncertainArray(zema_data.values[0][:2], zema_data.uncertainties[0][:2])
)
linear_inclusion = LinearInclusion(
    uncertain_inputs,
    Sigmoid,
    generate_weights_and_biases(
        len(uncertain_inputs.values),
        construct_out_features_counts(
            len(uncertain_inputs.values), depth=1
        ),
        seed=2,
    ),
)

In [24]:
optimization = RobustnessVerification(linear_inclusion)
solution: str = optimization.solve()
print(solution)

['x_0^(0): -1.1688331982839295', 'x_1^(0): -1.393796995951563', 'x_0^(1): 0.8581344384400857', 'x_1^(1): 0.4985904236486674', 'inf r_0^(1): 0.0', 'sup r_0^(1): 0.3581413720400519', 'inf r_1^(1): 0.0', 'sup r_1^(1): -0.0', '\n']presolving:
(round 1, fast)       4 del vars, 3 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       6 del vars, 3 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       6 del vars, 3 del conss, 0 add conss, 10 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       6 del vars, 3 del conss, 0 add conss, 12 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       7 del vars, 4 del conss, 0 add conss, 12 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 6, fast)       8 del vars, 5 del conss, 0 add conss, 12 chg bounds, 1 chg sides, 0 chg coeffs, 

### Extract and prepare linear inclusion parameters

In [83]:
def _construct_edge_idxs(
    x_coords: NDArray[np.double], y_coords: NDArray[np.double]
) -> NDArray[np.double]:
    """Build two-dimensional vector of x and y coordinates along the edges of input vectors

    The edges will be parallel to x and y axes.
    """
    assert_equal(len(x_coords), len(y_coords))
    return np.vstack(
        (
            np.concatenate(
                (
                    x_coords[x_coords == x_coords.min(initial=np.infty)].repeat(len(y_coords)),
                    x_coords[
                        np.logical_and(
                            x_coords != x_coords.min(initial=np.infty), x_coords != x_coords.max(initial=-np.infty)
                        )
                    ],
                    x_coords[x_coords == x_coords.max(initial=-np.infty)].repeat(len(y_coords)),
                    x_coords[
                        np.logical_and(
                            x_coords != x_coords.min(initial=np.infty), x_coords != x_coords.max(initial=-np.infty)
                        )
                    ],
                    x_coords[0:1],
                )
            ),
            np.concatenate(
                (
                    y_coords,
                    np.repeat([y_coords.max(initial=-np.infty)], len(x_coords) - 2),
                    y_coords[-1::-1],
                    np.repeat([y_coords.min(initial=np.infty)], len(x_coords) - 2),
                    y_coords[0:1],
                )
            ),
        )
    )

In [84]:
theta = dict()
theta_0_0 = linear_inclusion.theta[0][0][0]
theta_1_0 = linear_inclusion.theta[0][1][0]
theta_0 = _construct_edge_idxs(np.array(theta_0_0), np.array(theta_1_0))
theta_0_1 = linear_inclusion.theta[1][0][0]
theta_1_1 = linear_inclusion.theta[1][1][0]
theta_1 = _construct_edge_idxs(np.array(theta_0_1), np.array(theta_1_1))
z_0_1 = linear_inclusion.z_is[0][0][0]
z_1_1 = linear_inclusion.z_is[0][1][0]
xi_0_1 = linear_inclusion.xi_is[0][0]
xi_1_1 = linear_inclusion.xi_is[0][1]
xi_1 = np.vstack((xi_0_1, xi_1_1))
assert_equal(xi_1.shape, (2, 1))
if optimization.model.getSols():
    lower_r_0_1 = optimization.model.getVal(optimization.inf_r_is[1, 0])
    lower_r_1_1 = optimization.model.getVal(optimization.inf_r_is[1, 1])
    upper_r_i_0_1 = optimization.model.getVal(optimization.sup_r_is[1, 0])
    upper_r_i_1_1 = optimization.model.getVal(optimization.sup_r_is[1, 1])
else:
    lower_r_0_1 = linear_inclusion.r_is[0][0][0].inf
    lower_r_1_1 = linear_inclusion.r_is[0][1][0].inf
    upper_r_i_0_1 = linear_inclusion.r_is[0][0][0].sup
    upper_r_i_1_1 = linear_inclusion.r_is[0][1][0].sup
lower_r_1 = np.vstack((lower_r_0_1, lower_r_1_1))
assert_equal(lower_r_1.shape, (2, 1))
upper_r_1 = np.vstack((upper_r_i_0_1, upper_r_i_1_1))
assert_equal(upper_r_1.shape, (2, 1))
points_per_dim = 20
points_per_edges = 4 * (points_per_dim - 1) + 1
x_0 = _construct_edge_idxs(
    np.linspace(theta_0_0.inf, theta_0_0.sup, points_per_dim),
    np.linspace(theta_1_0.inf, theta_1_0.sup, points_per_dim),
)
assert_equal(x_0.shape, (2, points_per_edges))
x_1 = _construct_edge_idxs(
    np.linspace(theta_0_1.inf, theta_0_1.sup, points_per_dim),
    np.linspace(theta_1_1.inf, theta_1_1.sup, points_per_dim),
)
assert_equal(x_1.shape, (2, points_per_edges))

In [85]:
linear_inclusion.theta

((interval([-5.1606377858191035, 4.074591920636097]),
  interval([-1.393796995951563, 0.6986749809092526])),
 (interval([0.12899096008497263, 0.8581344384400857]),
  interval([0.048269106479496726, 0.9113382321424661])))

In [86]:
fig_edge = go.Figure()
fig_edge.add_trace(
    go.Scatter(x=x_0[0], y=x_0[1], mode="none", name="$\Large{x^{(0)}}$", fill="toself")
)
fig_edge.update_layout(
    title=dict(text="Inputs", **shared_title_params), **shared_layout
)
fig_edge.update_yaxes(**shared_ratio)
fig_edge.show()

In [87]:
linear_inclusion.nn_params.weights[0].shape

(2, 2)

In [88]:
x_0.shape

(2, 77)

In [89]:
linear_inclusion.nn_params.weights[0] @ x_0

array([[ 2.13701227,  2.10562777,  2.07424327,  2.04285877,  2.01147428,
         1.98008978,  1.94870528,  1.91732078,  1.88593628,  1.85455179,
         1.82316729,  1.79178279,  1.76039829,  1.72901379,  1.6976293 ,
         1.6662448 ,  1.6348603 ,  1.6034758 ,  1.57209131,  1.54070681,
         1.37683915,  1.21297149,  1.04910384,  0.88523618,  0.72136852,
         0.55750086,  0.39363321,  0.22976555,  0.06589789, -0.09796976,
        -0.26183742, -0.42570508, -0.58957274, -0.75344039, -0.91730805,
        -1.08117571, -1.24504336, -1.40891102, -1.57277868, -1.54139418,
        -1.51000968, -1.47862518, -1.44724069, -1.41585619, -1.38447169,
        -1.35308719, -1.32170269, -1.2903182 , -1.2589337 , -1.2275492 ,
        -1.1961647 , -1.1647802 , -1.13339571, -1.10201121, -1.07062671,
        -1.03924221, -1.00785771, -0.97647322,  1.97314461,  1.80927695,
         1.6454093 ,  1.48154164,  1.31767398,  1.15380633,  0.98993867,
         0.82607101,  0.66220335,  0.4983357 ,  0.3

In [90]:
z_1 = (
    (linear_inclusion.nn_params.weights[0] @ x_0).transpose()
    + linear_inclusion.nn_params.biases[0]
).transpose()
z_1 = np.concatenate((z_1[:,:58],np.flip(z_1[:,58:-1],axis=1),z_1[:,-1:]), axis=1)
assert z_1.shape == x_0.shape

In [91]:
h_of_z_1 = np.apply_along_axis(linear_inclusion.activation.func, 1, z_1)
assert h_of_z_1.shape == z_1.shape
h_of_z_1

array([[0.85813444, 0.85427059, 0.85031985, 0.84628129, 0.84215399,
        0.83793709, 0.83362979, 0.82923134, 0.82474106, 0.82015831,
        0.81548254, 0.81071326, 0.80585006, 0.80089259, 0.79584061,
        0.79069395, 0.78545252, 0.78011634, 0.7746855 , 0.76916023,
        0.73879362, 0.70595945, 0.67083686, 0.63369578, 0.59489465,
        0.5548705 , 0.51412171, 0.4731843 , 0.43260431, 0.39290889,
        0.35457966, 0.31803076, 0.28359386, 0.25151063, 0.2219326 ,
        0.19492712, 0.17048771, 0.14854707, 0.12899096, 0.13255833,
        0.13620892, 0.13994383, 0.14376411, 0.14767078, 0.1516648 ,
        0.1557471 , 0.15991857, 0.16418005, 0.1685323 , 0.17297604,
        0.17751195, 0.18214059, 0.18686252, 0.19167816, 0.19658791,
        0.20159205, 0.20669081, 0.21188429, 0.24053714, 0.27172875,
        0.30533894, 0.34115892, 0.37888906, 0.41814363, 0.45846321,
        0.49933417, 0.54021404, 0.58055984, 0.61985661, 0.65764281,
        0.69353007, 0.72721587, 0.75848845, 0.78

In [92]:
taylor_approx_1 = linear_inclusion.activation.func(
    xi_1
) + linear_inclusion.activation.deriv(xi_1) * (z_1 - xi_1)
assert taylor_approx_1.shape == z_1.shape
taylor_approx_1.shape, x_1.shape

((2, 77), (2, 77))

In [93]:
taylor_plus_lower_r_1 = taylor_approx_1 + lower_r_1
assert taylor_plus_lower_r_1.shape == taylor_approx_1.shape
lower_linear_1 = np.array(
    [x - taylor for taylor in taylor_plus_lower_r_1.T for x in x_1.T]
).T
assert_equal(lower_linear_1.shape, (2, np.square(points_per_edges)))

In [94]:
a, b = np.array([[1.0, 1.0, 1.0], [1.0, 2.0, 3.0]]), np.array([[2.0, 3.0], [1.0, 2.0]])
a, b, a.shape, b.shape

(array([[1., 1., 1.],
        [1., 2., 3.]]),
 array([[2., 3.],
        [1., 2.]]),
 (2, 3),
 (2, 2))

In [95]:
np.array([taylor - x for taylor in a.T for x in b.T]).T

array([[-1., -2., -1., -2., -1., -2.],
       [ 0., -1.,  1.,  0.,  2.,  1.]])

In [96]:
taylor_plus_upper_r_1 = taylor_approx_1 + upper_r_1
assert taylor_plus_upper_r_1.shape == taylor_approx_1.shape
upper_linear_1 = np.array(
    [x - taylor for taylor in taylor_plus_upper_r_1.T for x in x_1.T]
).T
assert lower_linear_1.shape == (2, np.square(points_per_edges))

## Visualize input region and discretization

In [97]:
fig_inputs = go.Figure()
fig_inputs.add_trace(
    go.Scatter(
        x=x_0[0], y=x_0[1], mode="none", name=r"$\Large{x^{(0)}}$", fill="toself"
    )
)
fig_inputs.add_trace(
    go.Scatter(x=theta_0[0], y=theta_0[1], mode="lines", name=r"$\Large{\Theta^{(0)}}$")
)
fig_inputs.update_layout(
    title=dict(text="Inputs", **shared_title_params), **shared_layout
)
fig_inputs.update_yaxes(**shared_ratio)
fig_inputs.show()

## Visualize first layer's transformations with discretizations

In [100]:
fig_1st_transforms = go.Figure()
fig_1st_transforms.add_trace(
    go.Scatter(
        x=theta_0[0],
        y=theta_0[1],
        mode="lines",
        name=r"$\Large{\Theta^{(0)}}$",
    )
)
fig_1st_transforms.add_trace(
    go.Scatter(x=theta_1[0], y=theta_1[1], mode="lines", name=r"$\Large{\Theta^{(1)}}$")
)
fig_1st_transforms.add_trace(
    go.Scatter(x=z_1[0], y=z_1[1], mode="none", name=r"$\Large{z^{(1)}}$",
        fill="toself",)
)
fig_1st_transforms.add_trace(
    go.Scatter(
        x=(z_0_1.inf, z_0_1.inf, z_0_1.sup, z_0_1.sup, z_0_1.inf),
        y=(z_1_1.inf, z_1_1.sup, z_1_1.sup, z_1_1.inf, z_1_1.inf),
        mode="lines",
        name=r"$\Large{Z^{(1)}}$",
    )
)
fig_1st_transforms.add_trace(
    go.Scatter(
        x=h_of_z_1[0],
        y=h_of_z_1[1],
        mode="none",
        name=r"$\Large{h (z^{(1)}})$",
        fill="toself",
    )
)
fig_1st_transforms.update_layout(
    title=dict(text="First layer's transformations", **shared_title_params),
    **shared_layout,
)
ytick_vals = np.linspace(z_1[1].min(), z_1[1].max(), 9)
assert isclose(ytick_vals[len(ytick_vals) // 2], xi_1_1)
ytick_labels = list(ytick_vals.round(1))
ytick_labels[len(ytick_labels) // 2] = r"$\Large{\xi_1^{(1)}}$"
xtick_vals = np.linspace(z_1[0].min(), z_1[0].max(), 9)
assert isclose(xtick_vals[len(xtick_vals) // 2], xi_0_1)
xtick_labels = list(xtick_vals.round(1))
xtick_labels[len(xtick_labels) // 2] = r"$\Large{\xi_0^{(1)}}$"
fig_1st_transforms.update_yaxes(
    **shared_ratio, tickvals=ytick_vals, ticktext=ytick_labels
)
fig_1st_transforms.update_xaxes(
    **shared_ratio, tickvals=xtick_vals, ticktext=xtick_labels
)
fig_1st_transforms.show()

In [105]:
fig_linear_inclusion = go.Figure()
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=theta_0[0],
        y=theta_0[1],
        mode="lines",
        name=r"$\Large{\Theta^{(0)}}$",
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=theta_1[0],
        y=theta_1[1],
        mode="lines",
        name=r"$\Large{\Theta^{(1)}}$",
    )
)
fig_linear_inclusion.add_trace(
    go.Scatter(x=z_1[0], y=z_1[1], mode="lines", name=r"$\Large{z^{(1)}}$")
)
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=lower_linear_1[0][lower_linear_1[0] <= 0],
        y=lower_linear_1[1][lower_linear_1[1] <= 0],
        mode="markers",
        name="lower bound",
        fill="none",
    )
)
# fig_linear_inclusion.add_trace(
#     go.Scatter(
#         x=upper_linear_1[0],
#         y=upper_linear_1[1],
#         mode="markers",
#         name="upper bound",
#         fill="toself",
#     )
# )
fig_linear_inclusion.add_trace(
    go.Scatter(
        x=(z_0_1.inf, z_0_1.inf, z_0_1.sup, z_0_1.sup, z_0_1.inf),
        y=(z_1_1.inf, z_1_1.sup, z_1_1.sup, z_1_1.inf, z_1_1.inf),
        mode="lines",
        name=r"$\Large{Z^{(1)}}$",
    )
)
fig_linear_inclusion.update_layout(
    title=dict(text="First layer's linear inclusion", **shared_title_params),
    **shared_layout,
)
fig_linear_inclusion.update_yaxes(**shared_ratio)
fig_linear_inclusion.show()